# Part I - Prosper Loan Data Exploration
### by Tristen Wallace

## Preliminary Wrangling Assessment

In [ ]:
# import all packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import custom_funcs

%matplotlib inline

In [ ]:
# Load dataset
file_path = '../data/prosperLoanData.csv'
df = pd.read_csv(file_path)

#Data structure
print(df.shape)

#Preview data
df.sample(5)


In [ ]:
# Review variables
df.info()

In [ ]:
# Review daterange
df.describe()


**Dimensionality Reduction:** We don't need many of these columns so we will create a subset with only the columns/features that interest us for this project.

In [ ]:
# Varaibles of interest
cols = ['LoanStatus', 'Term', 'CreditGrade', 'BorrowerAPR',
        'ProsperRating (Alpha)', 'ListingCategory (numeric)', 'BorrowerState', 'Occupation', 'EmploymentStatus',
        'IsBorrowerHomeowner', 'GroupKey', 'DebtToIncomeRatio', 'IncomeRange', 'ListingNumber', 'ProsperPrincipalBorrowed',
        'ProsperPrincipalOutstanding', 'LoanOriginalAmount', 'LoanOriginationDate', 'MemberKey', 'InvestmentFromFriendsCount', 
        'InvestmentFromFriendsAmount', 'Investors']

df_sub = df[cols]

# Descriptive statistics
df_sub.describe()


In [ ]:
# Rename variables
df_sub = df_sub.rename(columns={'ProsperRating (Alpha)':'ProsperRating',
                        'ListingCategory (numeric)':'ListingCategory'})

In [ ]:
# Check for missing values
df_sub.isnull().sum().sort_values(ascending=False)

In [ ]:
df.LoanOriginationDate = pd.to_datetime(df.LoanOriginationDate)
sns.heatmap(df_sub.set_index('LoanOriginationDate').sort_index().isna().transpose(),
            cbar_kws={'label':'Missing Data'});

In [ ]:
# Descriptive statistics of loans where at least one friend invested
df_sub.query('InvestmentFromFriendsCount > 1').describe() 


In [ ]:
# Check unique income ranges
df_sub.IncomeRange.value_counts()

In [ ]:
# Check counts for unique term lengths
df_sub.Term.value_counts()

In [ ]:
# Check counts for unique loan statuses
df_sub.LoanStatus.value_counts()

In [ ]:
# Check counts for unique employment statuses
df_sub.EmploymentStatus.value_counts()

In [ ]:
df_sub.query('EmploymentStatus == "Not available"')['IncomeRange']


In [ ]:
# Check unique occupation values
df_sub.Occupation.unique()

In [ ]:
# How many different occupations?
df_sub.Occupation.unique().size

In [ ]:
# Compare credit grade and Prosper rating
print(df_sub.ProsperRating.unique())
print(df_sub.CreditGrade.unique())

In [ ]:
df_sub.ListingCategory.value_counts().sort_values(ascending=False)

> Multiple variables contain placeholders for NaN values that avoided detection when searching for missing values. We should replace these values in `ListingCategory`, `IncomeRange`, and `EmploymentStatus` and reassess missing values visual.

In [ ]:
# Replace placeholders with NaN values
df_sub.ListingCategory = df_sub.ListingCategory.replace({0:np.nan})
df_sub.IncomeRange = df_sub.IncomeRange.replace({'Not displayed':np.nan})
df_sub.EmploymentStatus = df_sub.EmploymentStatus.replace({'Not available':np.nan})

# Visualize missing values
sns.heatmap(df_sub.set_index('LoanOriginationDate').sort_index().isna().transpose(),
            cbar_kws={'label':'Missing Data'});

> After assessing our variables of interest it's clear that we have several cleaning tasks to perform:

1. Our analysis doesn't require the granularity present in `LoanStatus`. We can combine the past due category and combine 'FinalPaymentInProgress' with 'Completed'.

2. Our analysis doesn't require the granularity present in `Occupation`. We can group occupations into smaller categories using ISCO grouping standards.

3. Based on the heatmap and their unique values comparison, we can see the `CreditGrade` and `ProsperRating` store the same variables. The two can be melted into one column using the ranking scheme AA - HR.

4. 19.4% of borrowers had exisiting loan balances with Prosper at the time of new loan origination. These pre-existing loan totals are stored in `ProsperPrincipalBorrowed` and `ProsperPrincipalOutstanding`. While in missing value assessment these variables appear to have 80.6% NaN values and qualify for removal, descriptive statistics show that the 19.4% of values have a meaningful value size to warrant keeping them. To deal with the missing values, we can engineer new 'TotalBorrowed' and 'TotalOutstanding' variables by combining the variables above with LoanOriginalAmount. 

5. Our analysis doesn't require the granularity present in `ListingCategory`. To reduce the categories, we'll keep the top 8 (1,7, 2, 3, 6, 4, 13, 15, 18), and lump the remaining in with 'Other'. 

6. `ListingCategory` is only missing values from before 08', but we can't drop these rows and it would be misleading to replace the NaN values with a common value. The best option is to leave the missing category as it was.

7. `IncomeRange` and `EmploymentStatus` weren't recorded prior to 07'. That date range isn't relavent to our analysis, so we can just drop the missing rows along with the remaining NaN values throughout the dataset

6. Remove unnecessary variables:
    - Group Key: Contains more than 70% NaN
    - IsBorrowerHomeowner, MemberKey: Not needed for our analysis


## Data Cleaning

In [ ]:
clean_df = df_sub.copy()

### 6. Remove unnecessary variables

In [ ]:
clean_df.drop(['GroupKey', 'IsBorrowerHomeowner', 'MemberKey'], axis=1, inplace=True)

### 2. Group Occupation Categories

In [ ]:
# Rename Occupation values to ISCO categories using custom mapp
for index, row in clean_df.iterrows():
    clean_df.at[index, 'Occupation'] = custom_funcs.map_occupation_to_isco(row['Occupation'])

In [ ]:
isco = ['Professionals', 'Clerical Support', 'Managers', 'Service/Sales',
        'Craft Workers', 'Other', 'Technicians', 'Skilled Agr/Forestry',
        'Operators/Assemblers', 'Armed Forces', 'Elementary']

# Test Occupations were converted
assert(all((clean_df.Occupation.value_counts().index == isco)))


### 1. Combine Loan Status Categories

In [60]:
clean_df.LoanStatus = clean_df.LoanStatus.replace({'Past Due (1-15 days)': 'Past Due',
                        'Past Due (31-60 days)': 'Past Due',
                        'Past Due (61-90 days)': 'Past Due',
                        'Past Due (91-120 days)': 'Past Due',
                        'Past Due (16-30 days)': 'Past Due',
                        'Past Due (>120 days)': 'Past Due',
                        'FinalPaymentInProgress': 'Completed'})

In [61]:
loan_statuses = ['Current', 'Completed', 'Chargedoff', 'Defaulted',
        'Past Due', 'Cancelled']

# Test loan statuses were converted
assert(all((clean_df.LoanStatus.value_counts().index == loan_statuses)))